In [18]:
import pandas as pd
import csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import utils

pd.options.mode.chained_assignment = None
%matplotlib inline

# K - Means

## Viendo info. que aporta

In [7]:
train_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
train_set = train_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses','price_aprox_usd']]


In [8]:
N_CLUSTERS = 8 ; MAX_ITER = 500
def print_results(centroids, num_cluster_points):
    print('\n\nFINAL RESULT:')
    for i, c in enumerate(centroids):
        print('\tCluster %d' % (i + 1))
        print('\t\tNumber Points in Cluster %d' % num_cluster_points.count(i))
        print('\t\tCentroid: %s' % str(centroids[i]))



# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=N_CLUSTERS, init="k-means++", n_init=10, max_iter=MAX_ITER, tol=0.0001, 
                precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(train_set)

# Obtenemos centro y numero de cluster para cada punto
centroids = kmeans.cluster_centers_
num_cluster_points = kmeans.labels_.tolist()
print_results(centroids, num_cluster_points)



FINAL RESULT:
	Cluster 1
		Number Points in Cluster 13903
		Centroid: [  1.38492812e+08   1.49910091e+00  -3.45568553e+01  -5.85267808e+01
   2.96869237e+02   1.64834928e+00   1.78975689e+02   1.22892397e+02
   1.03265482e+01   1.40868877e+00   2.88211177e+00   2.88593550e+04
   2.38510476e+05]
	Cluster 2
		Number Points in Cluster 9432
		Centroid: [  1.10581704e+08   1.58259118e+00  -3.44925866e+01  -5.85927125e+01
   3.05812871e+02   1.36842663e+00   2.14042515e+02   1.34501484e+02
   1.10660517e+01   1.49840967e+00   3.01558524e+00   2.93707159e+04
   2.55884573e+05]
	Cluster 3
		Number Points in Cluster 13887
		Centroid: [  1.22518373e+08   1.55332325e+00  -3.45456443e+01  -5.85112529e+01
   3.03965723e+02   1.67941240e+00   1.97703320e+02   1.32972204e+02
   1.05609563e+01   1.26391589e+00   2.93267084e+00   2.96892707e+04
   2.57855691e+05]
	Cluster 4
		Number Points in Cluster 4492
		Centroid: [  9.76609667e+07   1.61731968e+00  -3.44687052e+01  -5.86057217e+01
   2.95801425e+

In [9]:
dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}
dic_clusters

{0: array([27770, 27771, 27772, ..., 78654, 78655, 78656], dtype=int64),
 1: array([ 4141,  4142,  4143, ..., 50000, 50001, 50010], dtype=int64),
 2: array([10557, 10558, 10559, ..., 60502, 60503, 60504], dtype=int64),
 3: array([    0,     1,     2, ..., 38745, 38746, 38750], dtype=int64),
 4: array([16571, 16572, 16573, ..., 65954, 65955, 65956], dtype=int64),
 5: array([ 1364,  1365,  1366, ..., 42681, 42682, 42683], dtype=int64),
 6: array([ 7909,  7910,  7911, ..., 55006, 55007, 55008], dtype=int64),
 7: array([21259, 21260, 21261, ..., 71971, 71973, 71974], dtype=int64)}

## Probando hiper - parámetros

In [ ]:
for x in (200, 300, 500, 800):
    for z in (2,4,6,8,10,11,13, 15, 17, 19):
        total = 0
        cantidad = 0
        # Object KMeans
        kmeans = sklearn.cluster.KMeans(n_clusters=z, init="k-means++", n_init=10, max_iter=x, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

        # Calculate Kmeans
        kmeans.fit(train_set)

        dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}

        print("ANALISIS CON "+str(z)+" clusters y "+ str(x)+" COMO MAX ITERACION:")
        for i in range(z):
            puntos_cluster = dic_clusters[i]
            #Creo dataset auxiliar
            df = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses', 'price_aprox_usd', 'price_usd_per_m2'])
    
            for j in range(0,len(puntos_cluster)):
                df.loc[j] = train_set.loc[puntos_cluster[j]]
        
            df_to_predict = df[int(len(df)*0.7):]
            real_prediction = df_to_predict.price_aprox_usd
            df_to_predict = df_to_predict.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
            X_train = df[:int(len(df)*0.7)]
            Y_train = X_train.price_aprox_usd
            X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
    
            prediccion = linear_regression(df_to_predict, X_train, Y_train)
            precision = mean_squared_error(real_prediction, prediccion)
            print("Precision con el cluster "+ str(i) +":", precision )
            cantidad += precision
            total += 1
        
        print("PROMEDIO: "+ str(cantidad/total))

## K-means: predicciones por clusters

In [14]:
#Función a usar para predecir
def linear_regression(df_to_predict, x_train, y_train):
    scaler = StandardScaler()
    x_train_standarized =scaler.fit_transform(x_train)
    X_standarized = scaler.fit_transform(df_to_predict)
    regr = LinearRegression()
    regr.fit(x_train_standarized,y_train)
    y_price_usd_predicted = regr.predict(X_standarized)
    return y_price_usd_predicted


### K-means + lineal regression 

In [ ]:
#Hago K-means para el dataset de entrenamiento
train_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
train_set = train_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses','price_aprox_usd']]

# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=6, init="k-means++", n_init=10, max_iter=500, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(train_set)

dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}

#Hago K-means para el dataset de validación
predict_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
predict_set2 = predict_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses']]
predict_set_aux = predict_set2

import numpy as np
def setear_en_cero(row): return 0
#Columna que luego será llenada con las predicciones
predict_set_aux['prediccion'] = predict_set_aux.apply(lambda row: setear_en_cero(row),axis=1)

# Object KMeans
kmeans2 = sklearn.cluster.KMeans(n_clusters=6, init="k-means++", n_init=10, max_iter=500, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans2.fit(predict_set2)

dic_clusters_predict = {i: np.where(kmeans2.labels_ == i)[0] for i in range(kmeans2.n_clusters)}

#Una vez que se los clusters de cada dato extraigo del dataset de entrenamiento los datos del cluster 0 y los uso para predecir
#los datos del dataset de predicción pertenecientes también al cluster 0. Luego repito procedimiento para cada cluster.
for clave in dic_clusters:
    
    #Creación de datasets donde se almacenarán los datos de cada dataset pertenecientes a determinado cluster (según la iteración)
    df = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','price_aprox_usd'])
    df2 = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses'])
    
    for j in range(0,len(dic_clusters[clave])):
        df.loc[j] = train_set.loc[dic_clusters[clave][j]]
        
    print(dic_clusters_predict, len(dic_clusters_predict[clave]))
    for k in range(0,len(dic_clusters_predict[clave])):
        df2.loc[k] = predict_set2.loc[dic_clusters_predict[clave][k]]
    
    df_to_predict = df2.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
       
    X_train = df
    Y_train = X_train.price_aprox_usd
    X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]

    total_predicciones= linear_regression(df_to_predict, X_train, Y_train)
            
    i = 0

    for punto in dic_clusters_predict[clave]:
        if i < len(total_predicciones):
            predict_set_aux.loc[punto, 'prediccion'] = total_predicciones[i]
        i += 1
#Reabro el csv para obtener los ids        
predict_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
ids = predict_set.id
predicciones = predict_set_aux.prediccion
resultado = pd.DataFrame({'id': ids, 'price_usd' : predicciones})
resultado.to_csv("prediccion2.csv",encoding='utf-8',index=False)

## K-means + random forest regression

In [ ]:
#Hago K-means para el dataset de entrenamiento
train_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
train_set = train_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses','price_aprox_usd']]

# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=6, init="k-means++", n_init=10, max_iter=500, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(train_set)

dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}

#Hago K-means para el dataset de validación
predict_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
predict_set2 = predict_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses']]
predict_set_aux = predict_set2

import numpy as np
def setear_en_cero(row): return 0
#Columna que luego será llenada con las predicciones
predict_set_aux['prediccion'] = predict_set_aux.apply(lambda row: setear_en_cero(row),axis=1)

# Object KMeans
kmeans2 = sklearn.cluster.KMeans(n_clusters=6, init="k-means++", n_init=10, max_iter=500, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans2.fit(predict_set2)

dic_clusters_predict = {i: np.where(kmeans2.labels_ == i)[0] for i in range(kmeans2.n_clusters)}

#Una vez que se los clusters de cada dato extraigo del dataset de entrenamiento los datos del cluster 0 y los uso para predecir
#los datos del dataset de predicción pertenecientes también al cluster 0. Luego repito procedimiento para cada cluster.
for clave in dic_clusters:
    
    #Creación de datasets donde se almacenarán los datos de cada dataset pertenecientes a determinado cluster (según la iteración)
    df = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','price_aprox_usd'])
    df2 = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses'])
    
    for j in range(0,len(dic_clusters[clave])):
        df.loc[j] = train_set.loc[dic_clusters[clave][j]]
        
    print(dic_clusters_predict, len(dic_clusters_predict[clave]))
    for k in range(0,len(dic_clusters_predict[clave])):
        df2.loc[k] = predict_set2.loc[dic_clusters_predict[clave][k]]
    
    df_to_predict = df2.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
       
    X_train = df
    Y_train = X_train.price_aprox_usd
    X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]

    
    rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)
    rf.fit(X_train, Y_train)
    total_predicciones = rf.predict(df_to_predict)
     
    i = 0
    for punto in dic_clusters_predict[clave]:
        if i < len(total_predicciones):
            predict_set_aux.loc[punto, 'prediccion'] = total_predicciones[i]
        i += 1
#Reabro el csv para obtener los ids        
predict_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
ids = predict_set.id
predicciones = predict_set_aux.prediccion
resultado = pd.DataFrame({'id': ids, 'price_usd' : predicciones})
resultado.to_csv("prediccion2.csv",encoding='utf-8',index=False)

### K-means + KNN + Random Forest regression (ganador por ahora)

In [27]:
train_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
train_set = train_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses','price_aprox_usd']]

#Saco las ultimas dos filas por un tema de división del dataset en partes para el entrenamiento
train_set.drop(train_set.index[78656], inplace=True)
train_set.drop(train_set.index[78655], inplace=True)

df_to_predict = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
predic_set = df_to_predict.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]

train_set_aux = train_set.copy()

import numpy as np
def setear_en_cero(row): return 0
train_set_aux['knn'] = train_set_aux.apply(lambda row: setear_en_cero(row),axis=1)

puntos_division = len(train_set) // 5

z=0
j = puntos_division
total_pred = []
for x in range(5): 
    train_set_temporal = train_set[z:j]
    train_set_temporal = train_set_temporal.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
    frames = [train_set[:z], train_set[j:]]
    X_train = pd.concat(frames)
    X_train = X_train.astype(int)
    Y_train = X_train.price_aprox_usd
    X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
    neigh = KNeighborsClassifier(n_neighbors=100)
    neigh.fit(X_train, Y_train) 
    prediccion = neigh.predict(train_set_temporal)
    prediccion = prediccion.tolist()
    total_pred += prediccion
train_set_aux['knn'] = total_pred


#Clasifico con knn a los datos a predecir
predic_set = df_to_predict.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
X_train = train_set[:]
X_train = X_train.astype(int)
Y_train = X_train.price_aprox_usd
X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses',]]
neigh = KNeighborsClassifier(n_neighbors=15)
neigh.fit(X_train, Y_train) 
prediccion = neigh.predict(predic_set)
prediccion = prediccion.tolist()

predic_set['knn'] = prediccion



In [28]:
#Hago K-means para el dataset de entrenamiento


# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=6, init="k-means++", n_init=10, max_iter=500, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(train_set_aux)

dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}

#Hago K-means para el dataset de validación

predict_set_aux = predic_set

import numpy as np
def setear_en_cero(row): return 0
#Columna que luego será llenada con las predicciones
predict_set_aux['prediccion'] = predict_set_aux.apply(lambda row: setear_en_cero(row),axis=1)

# Object KMeans
kmeans2 = sklearn.cluster.KMeans(n_clusters=6, init="k-means++", n_init=10, max_iter=500, tol=0.0001, 
                        precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans2.fit(predict_set_aux)

dic_clusters_predict = {i: np.where(kmeans2.labels_ == i)[0] for i in range(kmeans2.n_clusters)}

#Una vez que se los clusters de cada dato extraigo del dataset de entrenamiento los datos del cluster 0 y los uso para predecir
#los datos del dataset de predicción pertenecientes también al cluster 0. Luego repito procedimiento para cada cluster.
for clave in dic_clusters:
    
    #Creación de datasets donde se almacenarán los datos de cada dataset pertenecientes a determinado cluster (según la iteración)
    df = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','price_aprox_usd'])
    df2 = pd.DataFrame(columns=['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses'])
    
    for j in range(0,len(dic_clusters[clave])):
        df.loc[j] = train_set_aux.loc[dic_clusters[clave][j]]
        
    
    for k in range(0,len(dic_clusters_predict[clave])):
        df2.loc[k] = predict_set_aux.loc[dic_clusters_predict[clave][k]]
    
    df_to_predict = df2.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]
       
    X_train = df
    Y_train = X_train.price_aprox_usd
    X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses']]

    rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)
    rf.fit(X_train, Y_train)
    total_predicciones = rf.predict(df_to_predict)
    
    i = 0
    for punto in dic_clusters_predict[clave]:
        if i < len(total_predicciones):
            predict_set_aux.loc[punto, 'prediccion'] = total_predicciones[i]
        i += 1
#Reabro el csv para obtener los ids        
predict_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
ids = predict_set.id
predicciones = predict_set_aux.prediccion
resultado = pd.DataFrame({'id': ids, 'price_usd' : predicciones})
resultado.to_csv("prediccion2.csv",encoding='utf-8',index=False)

# K - Means usando los clusters como una columna más 

## Usando linear regression 

In [12]:
train_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
train_set = train_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses','price_aprox_usd']]

N_CLUSTERS = 15 ; MAX_ITER = 500
def print_results(centroids, num_cluster_points):
    print('\n\nFINAL RESULT:')
    for i, c in enumerate(centroids):
        print('\tCluster %d' % (i + 1))
        print('\t\tNumber Points in Cluster %d' % num_cluster_points.count(i))
        print('\t\tCentroid: %s' % str(centroids[i]))



# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=N_CLUSTERS, init="k-means++", n_init=10, max_iter=MAX_ITER, tol=0.0001, 
                precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(train_set)
dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}
for clave,valor in dic_clusters.items():
    train_set.loc[valor, 'cluster'] = clave
train_set.head(3)

,created_on,property_type,lat,lon,place_name,state_name,surface_total_in_m2,surface_covered_in_m2,description,floor,rooms,expenses,price_aprox_usd,cluster
0,94953600.0,2,-35.050505,-59.710847,156,3,1132.0,251.0,20,1.0,3.0,23697.142857,440000.0,8.0
1,94953600.0,1,-34.603714,-58.381581,430,3,52.0,52.0,20,1.0,1.0,10.000000,125000.0,8.0
2,94953600.0,4,-34.628492,-58.390903,179,3,219.0,190.0,0,1.0,3.0,45000.000000,240000.0,8.0


In [15]:
X_train = train_set[:]
Y_train = X_train.price_aprox_usd
X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]

df_to_predict = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=N_CLUSTERS, init="k-means++", n_init=10, max_iter=MAX_ITER, tol=0.0001, 
                precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(df_to_predict)

#Creo la columna de los clusters para el ds con los datos a predecir
dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}
for clave,valor in dic_clusters.items():
    df_to_predict.loc[valor, 'cluster'] = clave
ids = df_to_predict.id

df_to_predict = df_to_predict.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]

prediccion = linear_regression(df_to_predict, X_train, Y_train)
resultado = pd.DataFrame({'id': ids, 'price_usd' : prediccion})
resultado.to_csv("prediccion2.csv",encoding='utf-8',index=False)

## Usando Random Forest regression 

In [16]:
rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)
rf.fit(X_train, Y_train)
predicted_test = rf.predict(df_to_predict)
resultado = pd.DataFrame({'id': ids, 'price_usd' : predicted_test})
resultado.to_csv("prediccion2.csv",encoding='utf-8',index=False)

## K - Means + KNN + Random Forest regression

In [21]:
#Repito proceso de asignarle a los datasets la columna con los clusters

train_set = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
train_set = train_set.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
       'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
       'rooms', 'expenses','price_aprox_usd']]

#Saco las ultimas dos filas por un tema de división del dataset en partes para el entrenamiento
train_set.drop(train_set.index[78656], inplace=True)
train_set.drop(train_set.index[78655], inplace=True)

N_CLUSTERS = 15 ; MAX_ITER = 500
def print_results(centroids, num_cluster_points):
    print('\n\nFINAL RESULT:')
    for i, c in enumerate(centroids):
        print('\tCluster %d' % (i + 1))
        print('\t\tNumber Points in Cluster %d' % num_cluster_points.count(i))
        print('\t\tCentroid: %s' % str(centroids[i]))



# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=N_CLUSTERS, init="k-means++", n_init=10, max_iter=MAX_ITER, tol=0.0001, 
                precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(train_set)

dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}
for clave,valor in dic_clusters.items():
    train_set.loc[valor, 'cluster'] = clave
    

X_train = train_set[:]
Y_train = X_train.price_aprox_usd
X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]

df_to_predict = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_predict.csv')
# Object KMeans
kmeans = sklearn.cluster.KMeans(n_clusters=N_CLUSTERS, init="k-means++", n_init=10, max_iter=MAX_ITER, tol=0.0001, 
                precompute_distances="auto", verbose=0, random_state=None, copy_x=True, n_jobs=1, algorithm="auto")

# Calculate Kmeans
kmeans.fit(df_to_predict)

dic_clusters = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}
for clave,valor in dic_clusters.items():
    df_to_predict.loc[valor, 'cluster'] = clave
ids = df_to_predict.id

#Copia donde tendré al dataset de entrenamiento con la columna de clusters y la clasificación por knn
train_set_aux = train_set.copy()

import numpy as np
def setear_en_cero(row): return 0
train_set_aux['knn'] = train_set_aux.apply(lambda row: setear_en_cero(row),axis=1)

puntos_division = len(train_set) // 5

z=0
j = puntos_division
total_pred = []
for x in range(5): 
    train_set_temporal = train_set[z:j]
    train_set_temporal = train_set_temporal.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]
    frames = [train_set[:z], train_set[j:]]
    X_train = pd.concat(frames)
    X_train = X_train.astype(int)
    Y_train = X_train.price_aprox_usd
    X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]
    neigh = KNeighborsClassifier(n_neighbors=100)
    neigh.fit(X_train, Y_train) 
    prediccion = neigh.predict(train_set_temporal)
    prediccion = prediccion.tolist()
    total_pred += prediccion
train_set_aux['knn'] = total_pred

In [22]:
#Clasifico con knn a los datos a predecir
predic_set = df_to_predict.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]
X_train = train_set[:]
X_train = X_train.astype(int)
Y_train = X_train.price_aprox_usd
X_train = X_train.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','cluster']]
neigh = KNeighborsClassifier(n_neighbors=15)
neigh.fit(X_train, Y_train) 
prediccion = neigh.predict(predic_set)
prediccion = prediccion.tolist()

predic_set['knn'] = prediccion

In [23]:
Y_train= train_set_aux.price_aprox_usd
X_train = train_set_aux.loc[:,['created_on', 'property_type', 'lat', 'lon', 'place_name', 'state_name',
               'surface_total_in_m2', 'surface_covered_in_m2', 'description', 'floor',
               'rooms', 'expenses','knn','cluster']]

#Predigo con random forest regressor

rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=0)
rf.fit(X_train, Y_train)
predicted_test = rf.predict(predic_set)
resultado = pd.DataFrame({'id': ids, 'price_usd' : predicted_test})
resultado.to_csv("prediccion4.csv",encoding='utf-8',index=False)

# Haciendo un dataset con bins para el intento fallido de random forests clasificador

In [3]:
ds_bins = pd.read_csv('Desktop/Datos/tp2/data_filled_ready_to_train.csv')
ds_bins.head(2)

,created_on,property_type,lat,lon,place_name,state_name,surface_total_in_m2,surface_covered_in_m2,description,floor,rooms,expenses,price_aprox_usd,price_usd_per_m2
0,94953600.0,2,-35.050505,-59.710847,156,3,1132.0,251.0,20,1.0,3.0,23697.142857,440000.0,388.692580
1,94953600.0,1,-34.603714,-58.381581,430,3,52.0,52.0,20,1.0,1.0,10.000000,125000.0,2403.846154


In [4]:
properati_bins = ds_bins.loc[ds_bins.price_usd_per_m2 > 2,:]
properati_bins = ds_bins.loc[ds_bins.price_usd_per_m2 < 61900.0,:]
properati_bins['bin'] = pd.qcut(properati_bins.price_usd_per_m2,10)
properati_bins.head(3)

,created_on,property_type,lat,lon,place_name,state_name,surface_total_in_m2,surface_covered_in_m2,description,floor,rooms,expenses,price_aprox_usd,price_usd_per_m2,bin
0,94953600.0,2,-35.050505,-59.710847,156,3,1132.0,251.0,20,1.0,3.0,23697.142857,440000.0,388.692580,"(226.922, 628.764]"
1,94953600.0,1,-34.603714,-58.381581,430,3,52.0,52.0,20,1.0,1.0,10.000000,125000.0,2403.846154,"(2250.0, 2526.316]"
2,94953600.0,4,-34.628492,-58.390903,179,3,219.0,190.0,0,1.0,3.0,45000.000000,240000.0,1095.890411,"(1000.0, 1304.348]"


In [5]:
properati_bins = properati_bins.loc[:,['FECHA','property_type','state_name','lat','lon','surface_total_in_m2','surface_covered_in_m2','bin']]
properati_bins = properati_bins[:14000]
properati_bins.to_csv("filtradobinsgrande.csv",encoding='utf-8',index=False)